In [4]:
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.vectorstores import Qdrant
import os
import pinecone
from tqdm.autonotebook import tqdm

In [2]:

os.environ["COHERE_API_KEY"] = "hv5YTaV6oUo5T9LGOY8F4bBtalGflhTU2FdPtEk3"
# initailize pinecode 

pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),  # find at app.pinecone.io
    environment=os.getenv("PINECONE_ENV"),  # next to api key in console
)

In [68]:
# load our document here 
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="./BankFAQs.csv", encoding='utf8')

data = loader.load()

In [69]:
data[:2]

[Document(page_content='Do I need to enter ‘#’ after keying in my Card number/ Card expiry date/ CVV number: What details are required when I want to perform a secure IVR transaction\nPlease listen to the recorded message and follow the instructions while entering your card details.: To perform a secure IVR transaction, you will need your 16-digit Card number, Card expiry date, CVV number, mobile number and IVR password.\nsecurity: security', metadata={'source': './BankFAQs.csv', 'row': 0}),
 Document(page_content='Do I need to enter ‘#’ after keying in my Card number/ Card expiry date/ CVV number: How should I get the IVR Password  if I hold an add-on card\nPlease listen to the recorded message and follow the instructions while entering your card details.: An IVR password can be requested only from the registered mobile number and will be sent to the registered mobile number / email ID of the primary card holder only.\nsecurity: security', metadata={'source': './BankFAQs.csv', 'row': 

In [87]:
# split texts into chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1200,
    chunk_overlap  = 200,
    length_function = len,
)

data_chunks = text_splitter.split_documents(data)

In [88]:
data_chunks[:3]

[Document(page_content='Do I need to enter ‘#’ after keying in my Card number/ Card expiry date/ CVV number: What details are required when I want to perform a secure IVR transaction\nPlease listen to the recorded message and follow the instructions while entering your card details.: To perform a secure IVR transaction, you will need your 16-digit Card number, Card expiry date, CVV number, mobile number and IVR password.\nsecurity: security', metadata={'source': './BankFAQs.csv', 'row': 0}),
 Document(page_content='Do I need to enter ‘#’ after keying in my Card number/ Card expiry date/ CVV number: How should I get the IVR Password  if I hold an add-on card\nPlease listen to the recorded message and follow the instructions while entering your card details.: An IVR password can be requested only from the registered mobile number and will be sent to the registered mobile number / email ID of the primary card holder only.\nsecurity: security', metadata={'source': './BankFAQs.csv', 'row': 

In [72]:
# initialize embeddings 
embeddings = CohereEmbeddings(model = "multilingual-22-12")

In [73]:
# vector storage

db = Qdrant.from_documents(data_chunks, embeddings, location=":memory:", collection_name="my_documents", distance_func="Cosine")

In [37]:
# index_name = "faqs-dbase"

# docsearch = Pinecone.from_documents(data_chunks, embeddings, index_name=index_name)

In [74]:
# make our prompt 
prompt_template = """

generate response to the question based on the text provided.

Change instances where the bank is HDFC Bank to GTbank, 

If the text doesn't contain the answer, reply that the answer is not available and can request for more assistance by dialing the hotline or sending a mail to customer service represntative.

the hotline is 0300104523 and the mail is customer@gtbank.com.gh

Text: {context}

Question: {question}
"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

In [75]:
question = "how do i create a bank account"

In [83]:
# question and answer here 
qa = RetrievalQA.from_chain_type(llm=Cohere(model="command-nightly", temperature=0), 
                                 chain_type="stuff", 
                                 retriever=db.as_retriever(search_type="mmr"),
                                                            
                                 chain_type_kwargs=chain_type_kwargs, 
                                 return_source_documents=True)
                                 

answer = qa({"query": question})

In [84]:
print(answer['result'])


 To create a bank account with GTbank, you will need to follow these steps:

1. Visit your nearest GTbank branch with the required documents, including proof of identity, proof of address, and a passport photo.

2. Fill out an application form with your personal information and choose the type of account you want to open.

3. Fund your account with the required minimum deposit.

4. Once your account has been created, you will be given a unique account number and a debit card.

It's important to note that you will need to keep your account active by performing at least one credit or debit transaction within a year to avoid any fees or penalties.

If you have any further questions or need more assistance, please feel free to contact our customer service team at 0300104523 or customer@gtbank.com.gh.


In [78]:
def question_and_answer(question):
    qa = RetrievalQA.from_chain_type(llm=Cohere(model="command-nightly", temperature=0), 
                                 chain_type="stuff", 
                                 retriever=db.as_retriever(search_type = "mmr"), 
                                 chain_type_kwargs=chain_type_kwargs, 
                                 return_source_documents=True)
                                 

    results = qa({"query": question})

    answer = results['result']

    return answer

In [80]:
print(question_and_answer("how do i create a bank account"))

 To create a bank account with GTbank, you will need to follow these steps:

1. Visit your nearest GTbank branch with the required documents, including proof of identity, proof of address, and a passport photo.

2. Fill out an application form with your personal information and choose the type of account you want to open.

3. Fund your account with the required minimum deposit.

4. Once your account has been created, you will be given a unique account number and a debit card.

It's important to note that you will need to keep your account active by performing at least one credit or debit transaction within a year to avoid any fees or penalties.

If you have any further questions or need more assistance, please feel free to contact our customer service team at 0300104523 or customer@gtbank.com.gh.
